In [3]:
from DataFetcher import DataFetcher
from USTs import USTs
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd
from datetime import datetime

In [4]:
auctions = DataFetcher().fetch_auction_data()
prices = DataFetcher().fetch_historical_UST_data(date=datetime(year=2025, month=6, day=17))

In [3]:
auctions

,cusip,auction_date,issue_date,security_term,maturity_date,avg_med_yield,currently_outstanding,original_security_term
0,91282CNJ6,2025-06-26,2025-06-30,7-Year,2032-06-30,null,null,7-Year
1,91282CMX6,2025-06-25,2025-06-27,1-Year 10-Month,2027-04-30,null,60044000000.000000,2-Year
2,91282CNK3,2025-06-25,2025-06-30,5-Year,2030-06-30,null,null,5-Year
3,91282CNL1,2025-06-24,2025-06-30,2-Year,2027-06-30,null,null,2-Year
4,912797MG9,2025-06-24,2025-06-26,6-Week,2025-08-07,null,207829000000.000000,52-Week
...,...,...,...,...,...,...,...,...
75,9127935W2,1980-01-02,1980-01-08,52-Week,1981-01-02,null,null,52-Week
76,9127933Y0,1979-12-28,1980-01-03,13-Week,1980-04-03,null,null,null
77,9127934U7,1979-12-28,1980-01-03,26-Week,1980-07-03,null,null,26-Week
78,912810CK2,1979-11-01,1979-11-15,30-Year,2009-11-15,10.440000,null,30-Year


In [7]:
ust_set = USTs(auction_data=auctions, price_data=prices).get_current_UST_set()

Merged auction and price data successfully
No missing or excess data
All CUSIPs are identical between DataFrames


In [9]:
from datetime import timedelta
ust_set['days'] = ust_set['Maturity date'] - ust_set['issue_date']

In [2]:
ust_set

NameError: name 'ust_set' is not defined

In [12]:
ust_set[ust_set['Security type'] == 'Note']

,Cusip,Security type,Rate,Maturity date,Buy,Sell,End of day,security_term,issue_date
50,912828XZ8,Note,2.750,2025-06-30,0.000000,99.93750,99.93750,7-Year,2018-07-02
51,912828ZW3,Note,0.250,2025-06-30,0.000000,99.84375,99.84375,5-Year,2020-06-30
52,91282CHL8,Note,4.625,2025-06-30,0.000000,100.00000,100.00000,2-Year,2023-06-30
53,91282CEY3,Note,3.000,2025-07-15,0.000000,99.87500,99.87500,3-Year,2022-07-15
54,91282CAB7,Note,0.250,2025-07-31,0.000000,99.50000,99.50000,5-Year,2020-07-31
...,...,...,...,...,...,...,...,...,...
287,91282CKQ3,Note,4.375,2034-05-15,100.109375,100.09375,100.34375,10-Year,2024-05-15
288,91282CLF6,Note,3.875,2034-08-15,96.203125,96.18750,96.46875,10-Year,2024-08-15
289,91282CLW9,Note,4.250,2034-11-15,98.859375,98.84375,99.12500,10-Year,2024-11-15
290,91282CMM0,Note,4.625,2035-02-15,101.687500,101.68750,101.96875,10-Year,2025-02-18
